<a href="https://colab.research.google.com/github/adnaneaabbar/car-detection/blob/master/Autonomous_Driving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's build a detection system for a self-driving car.

### "You Only Look Once" (YOLO) is a popular algorithm because it achieves high accuracy while also being able to run in real-time. This algorithm "only looks once" at the image in the sense that it requires only one forward propagation pass through the network to make predictions. After non-max suppression, it then outputs recognized objects together with the bounding boxes.

### Because the YOLO model is very computationally expensive to train, we will load pre-trained weights.


---
---


### To collect data, drive.ai mounted a camera to the hood of the car, which takes pictures of the road ahead every few seconds while it drives around, playend on a loop it creates a video-like animation.



In [4]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('car-detection/nb_images/road_video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<center>
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
</center>
""" % data_url)